In [1]:
import pandas as pd
import numpy as np

titanic = pd.read_csv('titanic.csv')

# Просмотр первых строк датасета
print("Исходные данные:")
print(titanic.head())
print("\nИнформация о датасете:")
print(titanic.info())

Исходные данные:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   

In [2]:
# Сохраняем исходное количество строк
initial_rows = titanic.shape[0]

# Удаление строк с пропущенными значениями
titanic_cleaned = titanic.dropna()

# Количество строк после удаления
cleaned_rows = titanic_cleaned.shape[0]

print(f"\nУдалено строк с пропусками: {initial_rows - cleaned_rows}")
print(f"Осталось строк: {cleaned_rows}")


Удалено строк с пропусками: 708
Осталось строк: 183


In [3]:
# Определение нечисловых столбцов
non_numeric_cols = titanic_cleaned.select_dtypes(exclude=['number']).columns

# Исключаем Sex и Embarked из списка на удаление
cols_to_drop = [col for col in non_numeric_cols if col not in ['Sex', 'Embarked']]

# Удаление выбранных столбцов
titanic_cleaned = titanic_cleaned.drop(columns=cols_to_drop)

print("\nСтолбцы после удаления нечисловых:")
print(titanic_cleaned.columns)


Столбцы после удаления нечисловых:
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked'],
      dtype='object')


In [4]:
# Перекодировка Sex: male -> 0, female -> 1
titanic_cleaned['Sex'] = titanic_cleaned['Sex'].map({'male': 0, 'female': 1})

# Перекодировка Embarked: S -> 1, C -> 2, Q -> 3
titanic_cleaned['Embarked'] = titanic_cleaned['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

print("\nПример данных после перекодировки:")
print(titanic_cleaned[['Sex', 'Embarked']].head())


Пример данных после перекодировки:
    Sex  Embarked
1     1         2
3     1         1
6     0         1
10    1         1
11    1         1


In [5]:
# Проверяем наличие столбца PassengerId
if 'PassengerId' in titanic_cleaned.columns:
    titanic_cleaned = titanic_cleaned.drop(columns=['PassengerId'])
    print("\nСтолбец PassengerId удален")
else:
    print("\nСтолбец PassengerId отсутствует в данных")

print("\nФинальные столбцы:")
print(titanic_cleaned.columns)


Столбец PassengerId удален

Финальные столбцы:
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')


In [9]:
# Расчет процента потерянных данных
initial_size = initial_rows * titanic.shape[1]  
final_size = cleaned_rows * titanic_cleaned.shape[1]  
lost_percentage = 100 * (initial_size - final_size) / initial_size

print(f"\nПроцент потерянных данных: {lost_percentage:.2f}%")
print(f"Исходный размер данных: {initial_size} значений")
print(f"Финальный размер данных: {final_size} значений")

titanic_cleaned.to_csv('titanic_processed.csv', index=False)
print("\nОбработанные данные сохранены в файл 'titanic_processed.csv'")


Процент потерянных данных: 86.31%
Исходный размер данных: 10692 значений
Финальный размер данных: 1464 значений

Обработанные данные сохранены в файл 'titanic_processed.csv'


In [13]:
print("\nИтоговый датафрейм:")
print(titanic_cleaned.head())
print("\nИнформация о финальном датафрейме:")
print(titanic_cleaned.info())


Итоговый датафрейм:
    Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
1          1       1    1  38.0      1      0  71.2833         2
3          1       1    1  35.0      1      0  53.1000         1
6          0       1    0  54.0      0      0  51.8625         1
10         1       3    1   4.0      1      1  16.7000         1
11         1       1    1  58.0      0      0  26.5500         1

Информация о финальном датафрейме:
<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  183 non-null    int64  
 1   Pclass    183 non-null    int64  
 2   Sex       183 non-null    int64  
 3   Age       183 non-null    float64
 4   SibSp     183 non-null    int64  
 5   Parch     183 non-null    int64  
 6   Fare      183 non-null    float64
 7   Embarked  183 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 12.9 KB
None


In [11]:
from sklearn.model_selection import train_test_split

try:
    df = pd.read_csv('titanic_processed.csv')
except:
    df = titanic_cleaned.copy()

# Разделение на признаки (X) и целевую переменную (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]} записей")
print(f"Размер тестовой выборки: {X_test.shape[0]} записей")

Размер обучающей выборки: 128 записей
Размер тестовой выборки: 55 записей


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Масштабирование данных (логистическая регрессия чувствительна к масштабу)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание и обучение модели
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [15]:
from sklearn.metrics import accuracy_score

# Предсказание на тестовых данных
y_pred = logreg.predict(X_test_scaled)

# Расчет точности
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность модели: {accuracy:.4f}")

# Вывод коэффициентов модели
print("\nКоэффициенты модели:")
for feature, coef in zip(X.columns, logreg.coef_[0]):
    print(f"{feature}: {coef:.4f}")


Точность модели: 0.7091

Коэффициенты модели:
Pclass: -0.4208
Sex: 1.5130
Age: -0.3927
SibSp: -0.2138
Parch: -0.3301
Fare: 0.2297
Embarked: 0.1046


In [17]:

X_train_no_embarked = X_train.drop('Embarked', axis=1)
X_test_no_embarked = X_test.drop('Embarked', axis=1)

# Масштабирование
X_train_no_embarked_scaled = scaler.fit_transform(X_train_no_embarked)
X_test_no_embarked_scaled = scaler.transform(X_test_no_embarked)

# Обучение новой модели
logreg_no_embarked = LogisticRegression(random_state=42, max_iter=1000)
logreg_no_embarked.fit(X_train_no_embarked_scaled, y_train)

# Оценка точности
y_pred_no_embarked = logreg_no_embarked.predict(X_test_no_embarked_scaled)
accuracy_no_embarked = accuracy_score(y_test, y_pred_no_embarked)

print(f"\nТочность без признака Embarked: {accuracy_no_embarked:.4f}")
print(f"Изменение точности: {accuracy - accuracy_no_embarked:.4f}")

from sklearn.model_selection import cross_val_score

# С признаком Embarked
scores_with = cross_val_score(logreg, X_train_scaled, y_train, cv=5)
# Без признака Embarked
scores_without = cross_val_score(logreg_no_embarked, X_train_no_embarked_scaled, y_train, cv=5)

print("\nРезультаты перекрестной проверки:")
print(f"Средняя точность с Embarked: {scores_with.mean():.4f} (±{scores_with.std():.4f})")
print(f"Средняя точность без Embarked: {scores_without.mean():.4f} (±{scores_without.std():.4f})")


Точность без признака Embarked: 0.6727
Изменение точности: 0.0364

Результаты перекрестной проверки:
Средняя точность с Embarked: 0.7652 (±0.0280)
Средняя точность без Embarked: 0.7809 (±0.0336)
